In [33]:
import datasets
import gzip
import json
import csv
import os

In [60]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)
    
def convert_to_json(gz_file_name, json_train_file_name, json_test_file_name, key_mapping = {'reviewText' : 'reviewText', 'overall' : 'overall'}, split_percent = 0.9):
    
    n_points = 0
    g = gzip.open(gz_file_name, 'rb')
    for l in g:
        n_points += 1
    print("n_points", n_points)
    lidx = 0
    
    f = open(json_train_file_name, 'w')
    for d in parse(gz_file_name):
        row = {}
        for key in key_mapping:
            if key_mapping[key] not in d:
                continue
                
            row[key] = d[ key_mapping[key] ]
        
        if lidx == int(split_percent * n_points):
            f.close()
            f = open(json_test_file_name, 'w')
            
        json_str = json.dumps(row)
        f.write(json_str + "\n")
        lidx += 1

    f.close()

    
def convert_to_json_good_reads(gz_file_name, json_train_file_name, json_test_file_name, n_train = 200000, n_test = 10000):
    
#     n_points = 0
#     g = gzip.open(gz_file_name, 'rb')
#     for l in g:
#         n_points += 1
#     print("n_points", n_points)
    lidx = 0
    nidx = 0
    pos_labels = 0
    neg_labels = 0
    f = open(json_train_file_name, 'w')
    write_negative = False
    for d in parse(gz_file_name):
        
        if nidx >= (n_train + n_test):
            print(nidx)
            break
        
        for review in d['review_sentences']:
            row = {
                'reviewText' : review[1],
                'label' : int(review[0])
            }
            if row['label'] == 1:
                json_str = json.dumps(row)
                f.write(json_str + "\n")
                nidx += 1
                write_negative = True
                pos_labels += 1
            else:
                if neg_labels <= pos_labels:
                    json_str = json.dumps(row)
                    f.write(json_str + "\n")
                    nidx += 1
                    write_negative = False
                    neg_labels += 1
                
            if nidx == n_train:
                f.close()
                f = open(json_test_file_name, 'w')
                print("Opened test file")
            
        lidx += 1
    print("Pos labels", pos_labels)
    f.close()
    

In [61]:
gz_file_name = "/data2/paarth/HuggingFaceDatasets/localfiles/goodreads_reviews_spoiler.json.gz"
goodreads_train = "/data2/paarth/HuggingFaceDatasets/localfiles/goodreads_train.json"
goodreads_test = "/data2/paarth/HuggingFaceDatasets/localfiles/goodreads_test.json"
convert_to_json_good_reads(gz_file_name, goodreads_train, goodreads_test)

Opened test file
210052
Pos labels 105069


In [32]:
convert_to_json("/data2/paarth/HuggingFaceDatasets/localfiles/Video_Games.json.gz", "/data2/paarth/HuggingFaceDatasets/localfiles/Video_Games_train.json", "/data2/paarth/HuggingFaceDatasets/localfiles/Video_Games_test.json")

n_points 2565349


In [13]:
dataset = datasets.load_dataset('json', data_files=["/data2/paarth/hftest/1.json".format(fidx) for fidx in range(1, 100)])

Using custom data configuration default


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [22]:
dataset = datasets.load_dataset('json', data_files={'train': ["/data2/paarth/hftest/combined.json"]})

Using custom data configuration default


Dataset json downloaded and prepared to /home/paarth/.cache/huggingface/datasets/json/default-833ebe74f2dc1c9e/0.0.0/fb88b12bd94767cb0cc7eedcd82ea1f402d2162addc03a37e81d4f8dc7313ad9. Subsequent calls will reuse this data.


In [17]:
dataset['train'][0]

{'text': 'Amazon,\nI am shopping for Amazon.com gift cards for Christmas gifts and am really so disappointed that out of five choices there isn\'t one that says "Merry Christmas" or mentions Christmas at all!  I am sure I am not alone in wanting a card that reflects the actual "holiday" we are celebrating. On principle, I cannot send a Amazon gift card this Christmas.  What\'s up with all the Political Correctness?  Bad marketing decision.\nLynn',
 'label': 1}